# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [4]:
weather_csv_file = "city_data.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()

,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,saint-augustin,51.2260,-58.6502,5.94,71.0,19.0,18.37,CA,1.672872e+09
1,rio gallegos,-51.6226,-69.2181,57.25,62.0,100.0,13.80,AR,1.672872e+09
2,eirunepe,-6.6603,-69.8736,74.91,97.0,100.0,2.59,BR,1.672872e+09
3,wewak,-3.5534,143.6268,82.04,75.0,91.0,4.83,PG,1.672872e+09
4,saiha,22.4833,92.9667,51.49,73.0,1.0,4.03,IN,1.672872e+09


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

map_plot = weather_df.hvplot.points(
    "lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
update_weather = weather_df



update_weather = update_weather.loc[(update_weather["Max Temp"] < 80) & (update_weather["Max Temp"] > 70)]


update_weather = update_weather.loc[update_weather["Wind Speed"] < 10]


update_weather = update_weather.loc[update_weather["Cloudiness"] == 0]


update_weather = update_weather.dropna()

update_weather

,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
165,vallenar,-28.5708,-70.7581,74.89,47.0,0.0,7.43,CL,1.672872e+09
215,carnarvon,-24.8667,113.6333,77.22,81.0,0.0,7.34,AU,1.672872e+09
284,ascension,30.2169,-90.9001,75.20,26.0,0.0,9.22,US,1.672872e+09
491,conde,-7.2597,-34.9075,78.26,78.0,0.0,5.75,BR,1.672872e+09


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
hotel_df = update_weather

hotel_df

,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
165,vallenar,-28.5708,-70.7581,74.89,47.0,0.0,7.43,CL,1.672872e+09
215,carnarvon,-24.8667,113.6333,77.22,81.0,0.0,7.34,AU,1.672872e+09
284,ascension,30.2169,-90.9001,75.20,26.0,0.0,9.22,US,1.672872e+09
491,conde,-7.2597,-34.9075,78.26,78.0,0.0,5.75,BR,1.672872e+09


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
hotel_df = update_weather

# params dictionary to update each iteration
params = {
    "radius": 10000,
    "types": "lodging",
    "key": geoapify_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 165: vallenar.
Missing field/result... skipping.
------------
Retrieving Results for Index 215: carnarvon.
Missing field/result... skipping.
------------
Retrieving Results for Index 284: ascension.
Missing field/result... skipping.
------------
Retrieving Results for Index 491: conde.
Missing field/result... skipping.
------------


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

map_plot_2 = hotel_df.hvplot.points(
    "lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [lng,Lat]   (City,Humidity,Country)